In [2]:
import os
import sys
from typing import Callable

this_module = sys.modules[__name__]

def get_input() -> list[str]:
    return get_lines_from_file('./input')

def get_test_input(idx) -> list[str]:
    if os.path.isfile(f'./test_input{idx}'):
        return get_lines_from_file(f'./test_input{idx}')
    else:
        return get_lines_from_file('./test_input')
    
def get_test_result(idx) -> int:
    return get_int_from_file(f'./test_result{idx}')

def get_solution_func(idx) -> Callable:
    return getattr(this_module, f'solution{idx}')

def get_lines_from_file(filepath) -> list[str]:
    with open(filepath) as f:
        return [line.strip('\n') for line in f.readlines()]

def get_str_from_file(filepath) -> str:
    with open(filepath) as f:
        return f.readline().strip('\n')

def get_int_from_file(filepath) -> int:
    with open(filepath) as f:
        return int(f.readline().strip())

def log_invocation(func):
    def logged_func(*args):
        res = func(*args)
        print(f'{func.__name__}({args}) -> {res}')
        return res
    return logged_func
    
def run_test(idx: int) -> bool:
    res = get_solution_func(idx)(get_test_input(idx))
    test_res = get_test_result(idx)
    
    if test_res == res:
        print(f'Your solution for part {idx} works!!! :) (on the test input, that is)')
        print(f'Let`s try it on the actual input now...')
        return True
    else:
        print(f'Your solution for part {idx} does not work yet. Keep going!')
        print(f'You`ve got {res}, but the correct test result is {test_res}')
        return False

def run_solution(idx: int):
    sol = get_solution_func(idx)(get_input())
    print(f'The solution for part {idx} is: {sol}')

def run():
    if run_test(1):
        run_solution(1)
        print('\nOn to part 2...\n')
        if run_test(2):
            run_solution(2)

In [3]:
from typing import Tuple
from concurrent.futures import ProcessPoolExecutor

def parse_map(mapping_lines: str) -> list[list[int]]:
    unordered_maps = [[int(elem) for elem in line.split()] for line in mapping_lines]
    return sorted(unordered_maps, key=lambda m: m[0])

def parse_input(input_lines: list[str]) -> Tuple[list[int], list[list[int]]]:
    initial_seeds = [int(seed) for seed in input_lines[0].split(':')[1].split()]

    maps = []
    empty_lines = [i for i in range(len(input_lines)) if input_lines[i] == '']
    for idx, empty_idx in enumerate(empty_lines[:-1]):
        maps.append(parse_map(input_lines[empty_idx+2:empty_lines[idx+1]]))
    maps.append(parse_map(input_lines[empty_lines[-1]+2:]))

    return initial_seeds, maps

def do_mapping(map: list[list[int]], key: int) -> int:
    result = None
    for mapping_rule in map:
        if key >= mapping_rule[1] and key < (mapping_rule[1] + mapping_rule[2]):
            result = mapping_rule[0] + (key - mapping_rule[1])
            break
    if result is None:
        result = key
    return result

def generate_seed_sequence(seeds: list[int]):
    i = 0
    while i < len(seeds):
        for s in range(seeds[i+1]):
            yield s + seeds[i]
        i += 2

def solution1(input: list[str]) -> int:
    initial_seeds, maps = parse_input(input)
    
    results = []

    for seed in initial_seeds:
        key = seed
        for m in maps:
            key = do_mapping(m, key)
        results.append(key)

    return min(results)

def solution2(input: list[str]) -> int:
    initial_seeds, maps = parse_input(input)
    print(sum([s for i, s in enumerate(initial_seeds) if i % 2 == 1]))

    initial_seeds = generate_seed_sequence(initial_seeds)
    
    def calculate_location(initial_seed):
        for idx, seed in enumerate(initial_seeds):
            key = seed
            for m in maps:
                key = do_mapping(m, key)
            if min_res is None or key < min_res:
                min_res = key
            if idx % 1000000 == 0:
                print(idx)

        return min_res

    min_res = None

    with ProcessPoolExecutor(max_workers=2) as executor:
        for res in executor.map(calculate_location, initial_seeds):


run()

Your solution for part 1 works!!! :) (on the test input, that is)
Let`s try it on the actual input now...
The solution for part 1 is: 157211394

On to part 2...

27
0
Your solution for part 2 works!!! :) (on the test input, that is)
Let`s try it on the actual input now...
1589455465
0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000


KeyboardInterrupt: 